# Downloads

In [1]:
#!pip install sentencepiece

# Imports

In [2]:
import os
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace 
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

# Setting up LLM

## Importing the .Env variable

In [3]:
HF_TOKEN=os.environ.get('HF_TOKEN')
huggingface_repo_id="mistralai/Mistral-7B-Instruct-v0.3"

keeping the `temperature` low makes is accurate and very concise in the answers

In [4]:
def load_llm(huggingface_repo_id):
    llm = HuggingFaceEndpoint(  
        repo_id=huggingface_repo_id, 
        temperature=0.5,
        huggingfacehub_api_token=HF_TOKEN,
        task='conversational'
    )
    return llm


# Connecting to LLM and creating a chain

## Setting the prompt template

In [5]:
custom_prompt_template = """
You are a The Last of Us Part 2 expert assistant. Use the provided context to answer the user's question accurately and comprehensively.

IMPORTANT GUIDELINES:
- Answer directly and concisely - no small talk or greetings
- If the context contains spoiler warnings (⚠️), include them in your response before revealing spoilers
- When relevant, include image paths exactly as shown in the context (e.g., "Image: ../Data/Images/...")
- For counting questions (how many weapons, characters, etc.), provide specific numbers and breakdowns
- If you don't know the answer from the given context, state "I don't have that information in the provided context"
- Stick strictly to the provided context - do not add external knowledge
- For weapon/item locations, include the specific chapter and area mentioned
- For safe codes, provide both the code and the method to find it

Context: {context}

Question: {question}

Answer:"""

In [6]:
def set_custom_prompt(custom_prompt_template):
    prompt= PromptTemplate(template=custom_prompt_template,
                          input_variables=["context", "question"])
    return prompt

## Loading the FAISS Database

In [7]:
embedding_model= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

In [8]:
FAISS_DB_PATH = 'FAISS Database/'

Set the `allow_dangerous_deserialization` parameter to `True` becuase we trust the `FAISS` database
and its contents inside it because we made it

In [9]:
db = FAISS.load_local(FAISS_DB_PATH,
                      embedding_model,
                      allow_dangerous_deserialization=True)

## Creating the QA Chain

In [10]:
chat_model = ChatHuggingFace(llm=load_llm(huggingface_repo_id))
qa_chain=RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={'k':3}),
    return_source_documents=True,
    chain_type_kwargs={'prompt':set_custom_prompt(custom_prompt_template)}
)

## Creating user input

In [11]:
user_query=input("Ask WLF ")
response=qa_chain.invoke({'query': user_query})
print("Result : ", response["result"])
print("Source Documents: ", response["source_documents"])

Ask WLF  how many weapons are there in the game?


Result :   There are 26 total weapons in The Last of Us Part 2: 4 handguns, 8 long guns, 6 throwables, 4 melee weapons, and 1 recovery item (Health Kit).
Source Documents:  [Document(id='e8ab59d8-2fbb-44c4-b5dd-f176ad73eaed', metadata={'producer': 'Skia/PDF m141 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'More Questions and Answers The last of us part 2', 'source': '..\\Cleaned Data\\More Questions and Answers The last of us part 2.pdf', 'total_pages': 8, 'page': 1, 'page_label': '2'}, page_content='found\n \nor\n \npicked\n \nup\n \nduring\n \ngameplay.\n  Q:  How  many  melee  weapons  can  you  find?    A:  There  are  4  dedicated  melee  weapons:  Machete,  Hammer,  Switchblade,  and  Shiv.   Q:  How  many  different  locations  does  the  game  visit?    A:  The  game  visits  4  major  geographic  locations:  Jackson  (Wyoming),  Seattle  (Washington),  \nSanta\n \nBarbara\n \n(California),\n \nand\n \nvarious\n \nsmaller\n \nlocations\n \nincluding\